In [1]:
import sys
sys.path.append('..')
import numpy as np
import mne

# from constants import SEEG_fs, AUDIO_fs
from utils import PatientLoader


In [2]:
!ls ../data

kh10_1_audio.npy	 kh2_2_words.npy	 kh6_1_sEEG.npy
kh10_1_channelNames.npy  kh3_1_audio.npy	 kh6_1_words.npy
kh10_1_noisyAudio.npy	 kh3_1_channelNames.npy  kh7_1_audio.npy
kh10_1_sEEG.npy		 kh3_1_noisyAudio.npy	 kh7_1_channelNames.npy
kh10_1_words.npy	 kh3_1_sEEG.npy		 kh7_1_noisyAudio.npy
kh1_1_audio.npy		 kh3_1_words.npy	 kh7_1_sEEG.npy
kh1_1_channelNames.npy	 kh4_1_audio.npy	 kh7_1_words.npy
kh1_1_noisyAudio.npy	 kh4_1_channelNames.npy  kh8_1_audio.npy
kh1_1_sEEG.npy		 kh4_1_noisyAudio.npy	 kh8_1_channelNames.npy
kh1_1_words.npy		 kh4_1_sEEG.npy		 kh8_1_noisyAudio.npy
kh2_1_audio.npy		 kh4_1_words.npy	 kh8_1_sEEG.npy
kh2_1_channelNames.npy	 kh5_1_audio.npy	 kh8_1_words.npy
kh2_1_noisyAudio.npy	 kh5_1_channelNames.npy  kh9_1_audio.npy
kh2_1_sEEG.npy		 kh5_1_noisyAudio.npy	 kh9_1_channelNames.npy
kh2_1_words.npy		 kh5_1_sEEG.npy		 kh9_1_noisyAudio.npy
kh2_2_audio.npy		 kh5_1_words.npy	 kh9_1_sEEG.npy
kh2_2_channelNames.npy	 kh6_1_audio.npy	 kh9_1_words.npy
kh2_2_noisyAudio.npy	 kh6_1_c

3

In [10]:
from scipy.signal import decimate
import pyworld as pw

fs = 48e3
target_fs = 16e3
frame_period = 10

audio = PatientLoader(2, session=2, data_dir='../data').get_audio()
downsampled = decimate(audio, int(fs / target_fs))
downsampled = np.ascontiguousarray(downsampled, dtype=np.float64)

f0, t = pw.harvest(x=downsampled,
                   fs=target_fs,
                   frame_period=frame_period)
sp = pw.cheaptrick(x=downsampled,
                   f0=f0,
                   temporal_positions=t,
                   fs=target_fs)
ap = pw.d4c(x=audio,
            f0=f0,
            temporal_positions=t,
            fs=target_fs)

encoded_ap = pw.code_aperiodicity(aperiodicity=ap,
                                  fs=target_fs)
f0 = f0.reshape(-1, 1)
vuv = (f0 > 0).astype(int)

features = np.concatenate((sp, encoded_ap, f0, vuv), axis=1)